test

In [1]:
from IPython.display import display, Markdown

with open("example.md", "r") as f:
    display(Markdown(f.read()))

## restaurant_search
- i'm looking for a place to eat
- I want to grab lunch
- I am searching for a dinner spot
- i'm looking for a place in the [north](location) of town
- show me [chinese](cuisine) restaurants
- show me [chines](cuisine:chinese) restaurants in the [north](location)
- show me a [mexican](cuisine) place in the [centre](location)
- i am looking for an [indian](cuisine) spot called olaolaolaolaolaola
- search for restaurants
- anywhere in the [west](location)
- anywhere near [18328](location)
- I am looking for [asian fusion](cuisine) food
- I am looking a restaurant in [29432](location)
- I am looking for [mexican indian fusion](cuisine)
- [central](location) [indian](cuisine) restaurant

In [2]:
from spacy_crfsuite.dataset import read_file

train_data = read_file("example.md")
train_data

[{'text': "i'm looking for a place to eat", 'entities': []},
 {'text': 'I want to grab lunch', 'entities': []},
 {'text': 'I am searching for a dinner spot', 'entities': []},
 {'text': "i'm looking for a place in the north of town",
  'entities': [{'start': 31,
    'end': 36,
    'value': 'north',
    'entity': 'location'}]},
 {'text': 'show me chinese restaurants',
  'entities': [{'start': 8,
    'end': 15,
    'value': 'chinese',
    'entity': 'cuisine'}]},
 {'text': 'show me chines restaurants in the north',
  'entities': [{'start': 8,
    'end': 14,
    'value': 'chinese',
    'entity': 'cuisine'},
   {'start': 34, 'end': 39, 'value': 'north', 'entity': 'location'}]},
 {'text': 'show me a mexican place in the centre',
  'entities': [{'start': 10,
    'end': 17,
    'value': 'mexican',
    'entity': 'cuisine'},
   {'start': 31, 'end': 37, 'value': 'centre', 'entity': 'location'}]},
 {'text': 'i am looking for an indian spot called olaolaolaolaolaola',
  'entities': [{'start': 20,
  

In [3]:
from spacy_crfsuite.crf_extractor import CRFExtractor
from spacy_crfsuite.dataset import create_dataset

train_dataset = create_dataset(train_data)
crf_extractor = CRFExtractor({"c1": 0.01, "c2": 0.06})
crf_extractor.train(train_dataset)

In [7]:
import spacy

from spacy_crfsuite.tokenizer import SpacyTokenizer
from spacy_crfsuite.dense_features import DenseFeatures

def build_spacy_message(nlp, doc, use_dense_features=False):
    tokenizer = SpacyTokenizer(nlp)
    message = {"doc": doc, "text": doc.text}

    tokens = tokenizer.tokenize(message, attribute="doc")
    tokenizer.add_cls_token(tokens)
    message["tokens"] = tokens

    if use_dense_features:
        dense_features = DenseFeatures(nlp)
        text_dense_features = dense_features(message, attribute="doc")
        if len(text_dense_features) > 0:
            message["text_dense_features"] = text_dense_features

    return message

nlp = spacy.blank("en")
doc = nlp("show mexican restaurents up north")

message = build_spacy_message(nlp, doc)
entities = crf_extractor.process(message)
print(message, entities)

{'doc': show mexican restaurents up north, 'text': 'show mexican restaurents up north', 'tokens': [<spacy_crfsuite.tokenizer.Token object at 0x11dcc9470>, <spacy_crfsuite.tokenizer.Token object at 0x11dcc9908>, <spacy_crfsuite.tokenizer.Token object at 0x11dcc9390>, <spacy_crfsuite.tokenizer.Token object at 0x11dcc9240>, <spacy_crfsuite.tokenizer.Token object at 0x11dcc92b0>, <spacy_crfsuite.tokenizer.Token object at 0x11dcc9828>]} [{'start': 5, 'end': 12, 'value': 'mexican', 'entity': 'cuisine', 'confidence': 0.46661000836822464}, {'start': 28, 'end': 33, 'value': 'north', 'entity': 'location', 'confidence': 0.7644682602370566}]


In [9]:
import spacy

from spacy_crfsuite import CRFEntityExtractor

nlp = spacy.blank('en')
pipe = CRFEntityExtractor(nlp, crf_extractor=crf_extractor)
nlp.add_pipe(pipe)

doc = nlp("show mexican restaurents up north")
for ent in doc.ents:
    print(ent.text, "--", ent.label_)

mexican -- cuisine
north -- location
